In [ ]:
#%pip install rasterstats
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterstats import zonal_stats
from rasterio.mask import mask
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
shapefile_path = "C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Data/research_area/Shape_files/SA_research_area/Southern_Alberta_research.shp"
output_path = "C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Data/research_area/Elevation/outputs"
tif_files = glob.glob("C:/Users/walimunige.rupasingh/OneDrive - University of Calgary/Documents/Reasearch/Data/research_area/Elevation/*.tif")

In [ ]:
# 1. Load Shapefile ===
gdf = gpd.read_file(shapefile_path)

In [ ]:
# 2. Merge the TIFs ===
src_files_to_mosaic = [rasterio.open(fp) for fp in tif_files]
mosaic, out_transform = merge(src_files_to_mosaic)

In [ ]:
# 3. Write the Merged Raster to Disk ===
mosaic_meta = src_files_to_mosaic[0].meta.copy()
mosaic_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_transform
})

merged_tif_path = "merged_dem.tif"
with rasterio.open(merged_tif_path, "w", **mosaic_meta) as dest:
    dest.write(mosaic)

In [ ]:
# 4. Compute Zonal Statistics ===
stats = zonal_stats(
    shapefile_path,
    merged_tif_path,
    stats=["min", "max", "mean", "std", "median", "count"],
    nodata=None,
    geojson_out=True
)

In [ ]:
# 5. Create GeoDataFrame with Stats ===
stats_gdf = gpd.GeoDataFrame.from_features(stats)

In [ ]:
# 6. Save to New Shapefile or CSV ===
output_shapefile_path = os.path.join(output_path, "SA_elevation_with_stats.shp")
output_dir = os.path.dirname(output_shapefile_path)
output_csv_path = os.path.join(output_dir, "SA_stats.csv")

# save the GeoDataFrame to a shapefile
stats_gdf.to_file(os.path.join(output_path, "SA_elevation_with_stats.shp"))  # Save as shapefile
stats_gdf.drop(columns="geometry").to_csv(output_csv_path, index=False)  # Also save as CSV

print("✅ Elevation statistics extracted and saved.")

In [ ]:
# Load shapefile with elevation stats
gdf = gpd.read_file("SA_elevation_with_stats.shp")  # or your output file

# Plot mean elevation
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(column='mean', cmap='terrain', legend=True, edgecolor='black', ax=ax)

ax.set_title("Mean Elevation per Polygon", fontsize=14)
ax.axis('off')
plt.tight_layout()
# Save the plot
plt.savefig(os.path.join(output_path, "mean_elevation_plot.png"), dpi=300)
plt.show()

In [ ]:
# Clip the merged DEM using the geometry from the shapefile
with rasterio.open(merged_tif_path) as src:
	# Use all geometries from the GeoDataFrame
	shapes = [feature["geometry"] for feature in stats]
	clipped_dem, clipped_transform = mask(src, shapes=shapes, crop=True)

In [ ]:
# Mask nodata using mosaic_meta or assume 0 is nodata if not set
nodata_value = mosaic_meta.get("nodata", 0)
clipped_data = np.where(clipped_dem[0] == nodata_value, np.nan, clipped_dem[0])

# Plot
plt.figure(figsize=(12, 8))
plt.imshow(clipped_data, cmap='terrain', vmin= 0, vmax=3000)  # Adjust vmin and vmax as needed
plt.colorbar(label="Elevation (m)")
plt.title("Elevation Across Shapefile Area")
plt.axis('off')
plt.tight_layout()
# Save the current figure as a PNG file in the output_path directory
plt.savefig(os.path.join(output_path, 'clipped_elevation_map.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Example: Split polygons into elevation classes and save separate shapefiles

# Define elevation bins (adjust as needed)
elevation_bins = [0, 500, 1000, 1500, 2000,2500, 3000]
elevation_labels = [f"{elevation_bins[i]}_{elevation_bins[i+1]}m" for i in range(len(elevation_bins)-1)]

# Ensure 'mean' column exists in stats_gdf
if 'mean' in stats_gdf.columns:
    stats_gdf['elev_class'] = pd.cut(stats_gdf['mean'], bins=elevation_bins, labels=elevation_labels, include_lowest=True)
    # Convert to string for shapefile output
    stats_gdf['elev_class'] = stats_gdf['elev_class'].astype(str)
    for label in elevation_labels:
        subset = stats_gdf[stats_gdf['elev_class'] == str(label)]
        if not subset.empty:
            out_path = os.path.join(output_dir, f"SA_elevation_{label}.shp")
            subset.to_file(out_path)
            print(f"Saved: {out_path}")
else:
    print("Column 'mean' not found in stats_gdf.")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# Plot mean elevation per polygon
gdf.plot(column='mean', cmap='terrain', legend=True, edgecolor='black', ax=axs[0])
axs[0].set_title("Mean Elevation per Polygon", fontsize=14)
axs[0].axis('off')

# Plot clipped DEM raster
im = axs[1].imshow(clipped_data, cmap='terrain', vmin=0, vmax=3000)
axs[1].set_title("Elevation Across Shapefile Area", fontsize=14)
axs[1].axis('off')
fig.colorbar(im, ax=axs[1], fraction=0.046, pad=0.04, label="Elevation (m)")

plt.tight_layout()
plt.show()



